In [1]:
# import pandas as pd
# import os
# import glob
# import concurrent.futures
# import time

# directory = 'C:\\Users\\Trieu Pham\\Desktop\\230105 Payment Summary - 2023-01-06T085450.177'
# directory2 = 'C:\\Users\\trieu.pham\\Desktop\\230105 Payment Summary - 2023-01-06T085450.177'
# cate_dir = 'C:\\Users\\Trieu Pham\\Desktop\\DataCate.xlsx'
# cate_dir2 = 'C:\\Users\\trieu.pham\\Desktop\\DataCate.xlsx'

# # list of Excel files to read data from
# if os.path.exists(os.path.join(directory, '230101 Payment Summary - 2023-01-03T093340.600.xlsx')):
#     excel_files = glob.glob(os.path.join(directory, '*.xlsx'))
# else:
#     excel_files = glob.glob(os.path.join(directory2, '*.xlsx'))
# try:
#     # use cols 0, Barcode; 2, Category Name
#     df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,2])
#     pass
# except FileNotFoundError:
#     df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,2])
#     pass
# # create an empty list to store the dataframes
# # dfs = [] sho

# # def excel2csv
# def excel2csv(file):
#     file_path = 'data.csv'
#     # Check if exist file_path
#     if not os.path.exists(file_path):
#         with open(file_path, 'w') as f:
#             pass

#     # Read the first line of the file
#     with open(file_path, 'r') as f:
#         first_line = f.readline().strip()
#     # Check if the first line contains the expected column names
#     has_headers = 'InvoiceID' in first_line
#     df = pd.read_excel(file, sheet_name='Sheet2', skiprows=2, usecols=[4,7])
#     # df2 Category
    
#     # merge with Category, on "Barcode"
#     df = df.merge(df2, on='Barcode', how='left')
    
#     # Check if the first line contains the expected column names
#     df.to_csv('data.csv', mode='a', header= not has_headers, index=False)


# # loop through the list of Excel files
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     [executor.submit(excel2csv, file) for file in excel_files]
#     # for file in excel_files:
#     #     executor.submit(excel2csv,file)

# # # concatenate the dataframes into a single dataframe
# # result = pd.concat(dfs)
# # # merge with Category, on "Barcode"
# # result = result.merge(df2, on='Barcode', how='left')
# # # export the resulting dataframe to a CSV file
# # result.to_csv('data.csv', index=False)


In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Sample data
df1 = pd.read_csv('data.csv')


# Get data from Invoice ID with Barcode
df1 = df1[['InvoiceID','Category Name']]

df1 = df1[df1.groupby('InvoiceID')['Category Name'].transform('size') > 1]
dataset_str = df1
dataset_str = df1.groupby('InvoiceID').agg({
    'Category Name': lambda x: ','.join(str(i) for i in x)
}).reset_index()

C:\Users\Trieu Pham\AppData\Local\Temp\ipykernel_9480\3129530308.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('data.csv')


In [3]:
# Transform the data into a list of lists (transactions) with TransactionEncoder
# create a TransactionEncoder object
data = dataset_str[['Category Name']]

data = data.values.tolist()

data = [x[0].split(',') for x in data]

te = TransactionEncoder()

# transform the data into a list of lists
transactions = te.fit(data).transform(data)
transactions = pd.DataFrame(transactions, columns=te.columns_)

transactions

,Accessories,Add-on item Cat,Babycare,Batteries,Biscuits,Bottled Water,Bread Offsite,Candy Category,Carbonated Drinks - CSD,Category Name,...,Supplement,Tea,Tofu,Toys,Vegetables Cat,Vegetarian/Organic food,Whole Milk,Wine,Yogurt,nan
0,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3682224,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3682225,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3682226,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3682227,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
# Calculate frequent itemsets using Apriori algorithm

frequent_itemsets = apriori(transactions, min_support=0.005, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.007402,(Add-on item Cat)
1,0.034007,(Biscuits)
2,0.168422,(Bottled Water)
3,0.051026,(Bread Offsite)
4,0.049357,(Candy Category)
...,...,...
178,0.008411,"(Tea, Snacks for Drinks)"
179,0.005122,"(Yogurt, Snacks for Drinks)"
180,0.005126,"(Sports Drinks, Tea)"
181,0.007723,"(Tea, Yogurt)"


In [5]:
# Generate association rules from the frequent itemsets

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Non-potato Chips),(Biscuits),0.085636,0.034007,0.006514,0.076062,2.236663,0.003601,1.045518
1,(Biscuits),(Non-potato Chips),0.034007,0.085636,0.006514,0.191540,2.236663,0.003601,1.130994
2,(Biscuits),(Potato Chips),0.034007,0.066608,0.005013,0.147410,2.213100,0.002748,1.094773
3,(Potato Chips),(Biscuits),0.066608,0.034007,0.005013,0.075261,2.213100,0.002748,1.044612
4,(Tea),(Biscuits),0.169312,0.034007,0.005154,0.030444,0.895218,-0.000603,0.996325
...,...,...,...,...,...,...,...,...,...
253,(Tea),(Sports Drinks),0.169312,0.038766,0.005126,0.030275,0.780974,-0.001438,0.991244
254,(Tea),(Yogurt),0.169312,0.075173,0.007723,0.045613,0.606767,-0.005005,0.969026
255,(Yogurt),(Tea),0.075173,0.169312,0.007723,0.102733,0.606767,-0.005005,0.925798
256,(Yogurt),(Whole Milk),0.075173,0.044735,0.005630,0.074897,1.674227,0.002267,1.032604


In [6]:
# Display the association rules with support, lift, and confidence

print(rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']])
rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']].to_csv('result.csv', index=False)

            antecedents         consequents   support      lift  confidence
0    (Non-potato Chips)          (Biscuits)  0.006514  2.236663    0.076062
1            (Biscuits)  (Non-potato Chips)  0.006514  2.236663    0.191540
2            (Biscuits)      (Potato Chips)  0.005013  2.213100    0.147410
3        (Potato Chips)          (Biscuits)  0.005013  2.213100    0.075261
4                 (Tea)          (Biscuits)  0.005154  0.895218    0.030444
..                  ...                 ...       ...       ...         ...
253               (Tea)     (Sports Drinks)  0.005126  0.780974    0.030275
254               (Tea)            (Yogurt)  0.007723  0.606767    0.045613
255            (Yogurt)               (Tea)  0.007723  0.606767    0.102733
256            (Yogurt)        (Whole Milk)  0.005630  1.674227    0.074897
257        (Whole Milk)            (Yogurt)  0.005630  1.674227    0.125857

[258 rows x 5 columns]
